# Temporal Difference Learning
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eleni-vasilaki/rl-notes/blob/main/notebooks/06_tdlearning.ipynb)

In previous lectures, we discussed a form of the Bellman equation, which links the action-value of a state–action pair to that of the next:

$$
q^\pi(s, a) = \mathbb{E}_\pi[R_{t+1} + \gamma q^\pi(s', a') \mid S_t = s, A_t = a]
$$

Here:
- $q^\pi(s, a)$ is the **action-value function** under policy $\pi$, giving the expected return starting from state $s$, taking action $a$, and thereafter following policy $\pi$, where $\pi(a \mid s)$ is the probability of selecting action $a$ in state $s$.
- $R_{t+1}$ is the **reward** received after taking action $A_t = a$ in state $S_t = s$.
- $s' = S_{t+1}$ is the **next state**, and $a' = A_{t+1}$ is the **next action**, selected under policy $\pi$.
- $\gamma \in [0,1]$ is the **discount factor**, controlling the importance of future rewards.
- The expectation $\mathbb{E}_\pi$ is taken over both the stochastic transitions and the action choices under policy $\pi$.

This form of the equation is crucial for reinforcement learning because it establishes that the action-values of a state–action pair can be learned by knowing the immediate reward and the value of the next state–action pair. This enables recursive evaluation of action-values. The expectation is necessary because, in general, both the reward and the next state are stochastic.

We will use a simplified notation from now on:

$$
q^\pi(s, a) = \mathbb{E}_\pi[R_{t+1} + \gamma q^\pi(s', a') \mid s,a]
$$

Another way to think of this equation is that the expected return from the current state–action pair should match the immediate reward plus the expected return from the next state–action pair, discounted by $\gamma$. This discount factor arises from the definition of the total expected return we seek to maximise.

The action-value function is parameterised by the policy $\pi$ because how we select actions affects the rewards we collect. For example, if we follow a random policy—as assumed in earlier exercises for simplicity—we cannot expect to obtain the maximum return. Since $q^\pi(s, a)$ represents the expected return under a given policy, and assuming we know the true action-values, a greedy policy would be optimal. In fact, an optimal policy is one that is better than or equal to all other policies in terms of expected return. We can write this as:

$$
q^*(s,a) = \max_\pi q^\pi(s,a)
$$

By setting $\pi = *$, we obtain the **Bellman optimality equation**:

$$
q^*(s, a) = \mathbb{E}[r + \gamma \max_{a'} q^*(s', a') \mid s, a]
$$

We will now follow the same methodology used in the bandit case. Specifically, we will define a loss function that allows us to estimate $Q(s,a)$—a parameterised function approximating $q(s,a)$—and derive appropriate learning rule updates.


## SARSA

We begin with the Bellman expectation equation for a given policy $\pi$:

$$
q^\pi(s, a) = \mathbb{E}_\pi \left[ R_{t+1} + \gamma q^\pi(s', a') \mid S_t = s, A_t = a \right]
$$

Rearranging, we obtain:

$$
\mathbb{E}_\pi \left[ R_{t+1} + \gamma q^\pi(s', a') - q^\pi(s, a) \mid S_t = s, A_t = a \right] = 0
$$

This equation expresses a condition that we would like our learning algorithm to approximate: the expected difference between the state-action value $q^\pi(s, a)$ of the current state and the observed return, composed by the immediate reward $ R_{t+1}$ plus the discounted state-action value of the next state $ \gamma q^\pi(s', a')$ should be zero.

However, we do not know the true action-values $q^\pi$. We, therefore, introduce a parameterised function $Q(s, a)$ to estimate them. The expectation then becomes approximately zero:

$$
\mathbb{E}_\pi \left[ R_{t+1} + \gamma Q(s', a') - Q(s, a) \mid S_t = s, A_t = a \right] \approx 0
$$

We define a squared loss function around this quantity in order to make it minimisable. The key idea is that minimising the loss function will provide us with a approximation

Suppose we have observed $N$ transitions starting from $(s, a)$:

$$
(s, a, r^{(i)}, s'^{(i)}, a'^{(i)}), \quad i = 1, \dots, N
$$

The loss function becomes:

$$
\mathcal{L}(s,a) = \frac{1}{2N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma Q(s'^{(i)}, a'^{(i)}) \right] \right)^2
$$

We now take the partial derivative of the loss with respect to $Q(s,a)$:

$$
\frac{\partial \mathcal{L}(s,a)}{\partial Q(s,a)} = \frac{1}{N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma Q(s'^{(i)}, a'^{(i)}) \right] \right)
$$

In writing this, we assume that $Q(s'^{(i)}, a'^{(i)})$ is known and fixed. To update $Q(s, a)$, we apply:

$$
\Delta Q(s, a) = -\eta \frac{\partial \mathcal{L}(s,a)}{\partial Q(s,a)}
$$

Substituting:

$$
\Delta Q(s, a) = -\eta \frac{1}{N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma Q(s'^{(i)}, a'^{(i)}) \right] \right)
$$

In an online setting (i.e., $N = 1$), we define the instantaneous update for a single transition $(s, a, r, s', a')$:

$$
\Delta Q(s, a) = -\eta \left( Q(s, a) - \left[ r + \gamma Q(s', a') \right] \right)
$$

This difference between the observed return and the current estimate is often called the **temporal difference**, or **TD error**:

$$
\delta = r + \gamma Q(s', a') - Q(s, a)
$$

The update can then be written as:

$$
\Delta Q(s, a) = \eta \, \delta
$$

which is the SARSA update rule:

$$
\Delta Q(s, a) = \eta \left( r + \gamma Q(s', a') - Q(s, a) \right)
$$

SARSA stands for **State–Action–Reward–State–Action**, a name that reflects the elements involved in its update. It is an example of a **Temporal Difference (TD) Learning** algorithm, meaning it is based on bootstrapping from the Bellman equation. SARSA is an **on-policy** method: it updates using the action that was actually taken according to the current policy.


## The SARSA Algorithm

The following algorithm corresponds to the SARSA method. An **episode** consists of a complete "game", from initialising the agent in some state of the environment until it reaches a terminal state. An episode involves several **steps**, where the agent takes actions and transitions to new states. There is no next state after a terminal state, hence we take $Q(s', a') = 0$ in that case.

1. Initialise $Q(s, a)$ arbitrarily for all $s \in S$ and $a \in A(s)$.
2. Repeat (for each episode):
   - **a.** Initialise state $s$.
   - **b.** Choose action $a$ from $s$ using a policy derived from $Q$ (e.g., $\epsilon$-greedy).
   - **c.** Repeat (for each step of the episode):
       - **i.** Take action $a$, observe reward $r$ and next state $s'$.
       - **ii.** If $s'$ is terminal, update $Q(s, a)$: $Q(s, a) \leftarrow Q(s, a) + \eta \cdot [r - Q(s, a)]$ and break.
       - **iii.** Choose action $a'$ from $s'$ using a policy derived from $Q$ (e.g., $\epsilon$-greedy).
       - **iv.** Update $Q(s, a)$ using the rule: $Q(s, a) \leftarrow Q(s, a) + \eta \cdot \left[ r + \gamma \cdot Q(s', a') - Q(s, a) \right]$
       - **v.** Update state and action: $s \leftarrow s'$, $a \leftarrow a'$.
   - **d.** Continue until $s$ is terminal.

# Exercise

Consider a robot navigating a linear track. The robot is initially placed at the upper end of the maze (state $s_1$), while the reward is available at the other end (state $s_4$). This scenario can be modelled as a linear sequence of states with a unique reward $R = 1$ when the goal is reached. For each state (except the first and the last), the possible actions are going **forward** (towards $s_4$) and **backward** (towards $s_1$). When the goal is reached, the robot is placed back in the initial position $s_1$, and the exploration starts again.

Use the SARSA method, $\Delta Q(s,a) = \eta \left[ r - Q(s,a) + \gamma Q(s',a') \right]$, with $\eta = 1$ and $\gamma = 1$ for simplicity. Choose actions **greedily** (i.e., select the action with the highest Q-value; if all Q-values are equal, choose randomly). Initially, all Q-values are set to zero.

**Questions:**
- How do the Q-values develop as the robot walks down the maze once? Twice? Three or more times?
- What happens to the learning speed if the number of states increases?

<details>
<summary>Show Solution</summary>

For the SARSA method, the update rule is given by:  
$$\Delta Q(s, a) = \eta [r + \gamma Q(s', a') - Q(s, a)],$$  
where $\eta = 1$ and $\gamma = 1$ for this task. Initially, all Q-values are set to 0. We'll explore the updates after each walk down the maze.

**After taking the first walk (episode 1):**  
The robot moves from $s_3$ to $s_4$, taking action $a_1$, which leads directly to the reward. The SARSA update for this action, since $Q(s', a') = 0$ (as $s_4$ is terminal and resets the scenario), and $r = 1$, is:
$$\Delta Q(s_3, a_1) = 1 \cdot [1 + 1 \cdot 0 - 0] = 1.$$
Thus, $Q(s_3, a_1) = 1$ after this update.

**After taking the second walk (episode 2):**  
Now that the robot knows moving from $s_3$ to $s_4$ yields a reward, when moving from $s_2$ to $s_3$ (choosing $a_1$ again), we apply the update with $r = 0$ (no immediate reward for this action), but since $Q(s_3, a_1) = 1$, the update becomes:
$$\Delta Q(s_2, a_1) = 1 \cdot [0 + 1 \cdot 1 - 0] = 1.$$
Therefore, $Q(s_2, a_1) = 1$ after the robot's second walk.

**After taking the third walk (episode 3):**  
The knowledge of the reward propagates back to the starting position. From $s_1$ to $s_2$ with action $a_1$, the update uses $Q(s_2, a_1) = 1$ from the previous step, and $r = 0$:
$$\Delta Q(s_1, a_1) = 1 \cdot [0 + 1 \cdot 1 - 0] = 1.$$
Hence, $Q(s_1, a_1) = 1$ following the third walk.

After these updates, the Q-values accurately predict the path to the reward, leading to no further changes for these actions due to the greedy selection strategy: the robot always chooses $a_1$ from each state to move towards the goal.

**Consideration of Additional States:**  
If we added $n$ more states, extending the sequence, the propagation of reward knowledge through the Q-values would require one additional walk for each new state added. This is because the SARSA algorithm updates Q-values based on direct experiences and the learned value of subsequent actions, necessitating a sequential "backpropagation" of the reward information through the state space.

This solution illustrates the application of the SARSA update rule to a simple environment, highlighting the iterative nature of learning and the convergence behaviour as the agent gains experience.

</details>

## Tabular Eligibility Traces

Tabular eligibility traces in reinforcement learning (RL) bridge the gap between actions and their delayed rewards, allowing algorithms to assign credit not only to the immediate action but also to those that preceded it. As you saw in the "robot navigating a linear track" example, learning all the Q-values required several repetitions. It is possible to allow for a trace that marks the entire trajectory taken and informs earlier Q-values about the reward the robot found further down its path.

We implement a variable $e(s,a)$—akin to a memory that fades over time—known as the **eligibility trace** for each state–action pair $(s, a)$.

The update process for $e(s, a)$ at each step of an episode is as follows:

1. For the state–action pair $(s, a)$ just visited, increment the trace:
   $$
   e(s, a) \leftarrow e(s, a) + 1
   $$

2. For all state–action pairs, decay the eligibility trace by a factor of $\gamma \lambda$, where $\gamma$ is the discount factor and $\lambda$ is the trace decay parameter:
   $$
   e(s, a) \leftarrow \gamma \lambda e(s, a)
   $$

This process ensures that the influence of a state–action pair on learning decreases over time unless it is revisited and reinforced—analogous to how the strength of an ant’s pheromone trail decreases unless reinforced by subsequent ants following the same path. The decay rate of eligibility traces plays a crucial role in determining how long past actions influence future learning, much like the evaporative nature of pheromones in guiding ant foraging behaviour by emphasizing recent, successful paths.

SARSA and other reinforcement learning algorithms can incorporate the concept of eligibility traces to speed up convergence. A SARSA algorithm that uses eligibility traces is known as the **SARSA($\lambda$) algorithm**—similar naming conventions apply to other TD-learning algorithms.

### SARSA($\lambda$) Algorithm

1. **Initialise** $Q(s, a)$ arbitrarily for all $s \in S$, $a \in A(s)$.
2. **Repeat** for each episode:
   - **a.** For all $s, a$, reset eligibility traces: $e(s, a) = 0$.
   - **b.** Initialise state $s$.
   - **c.** Choose action $a$ from $s$ using a policy derived from $Q$ (e.g., $\epsilon$-greedy).
   - **d.** Repeat (for each step of the episode):
       - **i.** Take action $a$, observe reward $r$ and next state $s'$.
       - **ii.** Choose $a'$ from $s'$ using a policy derived from $Q$.
       - **iii.** If $s'$ terminal compute TD error as $\delta = r - Q(s, a)$ else $\delta = r + \gamma Q(s', a') - Q(s, a).$
       - **iv.** $e(s, a) = e(s, a) + 1$
       - **v.** For all $s, a$:
           - Update Q-value: $Q(s, a) \leftarrow Q(s, a) + \eta \delta e(s, a).$
           - Update eligibility trace: $e(s, a) \leftarrow \gamma \lambda e(s, a.)$
       - **vi.** $s \leftarrow s'; \quad a \leftarrow a'.$
   - **e.** Until $s$ is terminal.

# Exercise

Consider the robot of the previous scenario but now use the TD(λ) algorithm. For each state $s$ and action $a$, an eligibility trace $e(s, a)$ is stored. At each time step, all traces are updated by $e(s,a) = \lambda e(s,a)$, except for the trace corresponding to the current state $s^*$ and action $a^*$ for which $e(s^*,a^*) = \lambda e(s^*,a^*) + 1$. The TD(λ) update rule is $\Delta Q(s,a) = \eta [r - Q(s,a) + \gamma Q(s',a')] e(s,a)$, applied to **ALL** states and actions.

- Compared to the SARSA method, does the information propagate more rapidly with the TD(λ) algorithm?
- Does the number of computational operations change with the introduction of the TD(λ) algorithm?


<details>
<summary>Show Solution</summary>

Applying the TD(λ) algorithm with $\lambda=1$ and $\eta=1$, and assuming all Q-values are initially set to 0, let's analyze the learning process during a single walk through the maze from $s_1$ to $s_4$:

1. **Walk Through the Maze**: As the robot progresses from $s_1$ to $s_4$, the eligibility traces for the state-action pairs along the path are updated. Upon reaching $s_4$ and receiving the reward, the eligibility traces for these actions are 1.

2. **TD(λ) Update at Reward**: At the reward time, since $\lambda=1$, the update rule $\Delta Q(s,a) = \eta [r - Q(s,a) + \gamma Q(s',a')] e(s,a)$ is applied to all state-action pairs. Given that $e(s, a)$ is 1 for the path taken, the Q-values for these actions are updated in a single step to reflect the received reward.

3. **Information Propagation**: This method allows for the immediate and full update of the Q-values for the taken path in a single walk, showcasing a much more rapid propagation of reward information back through the state-action pairs compared to the iterative approach required by SARSA.

4. **Computational Operations**: Although the learning happens in a single walk, the computation involves updating Q-values for all state-action pairs, maintaining the same number of operations. However, the parallelizability of this approach (updating all pairs based on their eligibility traces) represents a significant advantage in systems capable of parallel processing.

In summary, the TD(λ) algorithm, particularly with $\lambda=1$, enables rapid information propagation and learning in a single episode. This demonstrates the efficiency of eligibility traces in spreading the reward signal back through the visited state-action pairs. While the computational complexity per step does not decrease, the ability to parallelize updates makes TD(λ) especially powerful in suitable computational environments.

</details>

## Q-learning

Q-learning is an algorithm that closely resembles SARSA, but it is motivated by the following form of the Bellman optimality equation:

$$
q^*(s, a) = \mathbb{E}[r + \gamma \max_{a'} q^*(s', a') \mid s, a]
$$

Here, the $\max_{a'}$ operator means that the agent uses the value of the best possible action in the next state $s'$, regardless of which action $a'$ it actually took.

Its update rule is given by:

$$
\Delta Q = \eta \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right)
$$

Q-learning is an off-policy algorithm: it updates using the best estimated value of the next state, independent of the action that was actually taken.

### The Q-learning Algorithm

1. Initialise $Q(s, a)$ arbitrarily for all $s \in S$ and $a \in A(s)$.
2. Repeat (for each episode):
   - **a.** Initialise state $s$.
   - **b.** Repeat (for each step of the episode):
       - **i.** Choose an action $a$ from $s$ using a policy derived from $Q$ (e.g., $\epsilon$-greedy).
       - **ii.** Take action $a$, observe reward $r$ and next state $s'$.
       - **iii.** If $s'$ is terminal, update $Q(s, a)$: $Q(s, a) \leftarrow Q(s, a) + \eta \cdot [r - Q(s, a)]$ and break.
       - **iv.** Update $Q(s, a)$: $Q(s, a) \leftarrow Q(s, a) + \eta \cdot [r + \gamma \cdot \max_{a'} Q(s', a') - Q(s, a)]$
       - **v.** Update state: $s \leftarrow s'$
   - **c.** Continue until $s$ is terminal.

Q-learning also has a version that incorporates eligibility traces, known as the Q-learning($\lambda$) algorithm.



### Q-learning($\lambda$) Algorithm

1. **Initialise** $Q(s, a)$ arbitrarily for all $s \in S$, $a \in A(s)$.
2. **Repeat** for each episode:
   - **a.** For all $s, a$, reset eligibility traces: $e(s, a) = 0$.
   - **b.** Initialise state $s$.
   - **c.** Repeat (for each step of the episode):
       - **i.** Choose action $a$ from $s$ using a policy derived from $Q$ (e.g., $\epsilon$-greedy).
       - **ii.** Take action $a$, observe reward $r$ and next state $s'$.
       - **iii.** If $s'$ is terminal compute TD error as $\delta = r - Q(s, a)$ else $\delta = r + \gamma \max_{a'} Q(s', a') - Q(s, a)$
       - **iv.** Update eligibility trace: $e(s, a) \leftarrow e(s, a) + 1$
       - **v.** For all $s, a$:
           - Update Q-value: $Q(s, a) \leftarrow Q(s, a) + \eta \delta e(s, a)$
           - Decay eligibility trace: $e(s, a) \leftarrow \gamma \lambda e(s, a)$
       - **vi.** Update state: $s \leftarrow s'$
   - **d.** Until $s$ is terminal.


# Exercise
Starting from the bellman equation for the expectation of action-values:

$$
q^*(s, a) = \mathbb{E} \left[ R_{t+1} + \gamma \max_{a'} q^*(s', a') \mid S_t = s, A_t = a \right]
$$

define an appropriate loss function and derive the Q-learning update rule by minimising this loss.

<details>
<summary>Show Solution</summary>

We begin with the Bellman optimality equation:

$$
q^*(s, a) = \mathbb{E} \left[ R_{t+1} + \gamma \max_{a'} q^*(s', a') \mid S_t = s, A_t = a \right]
$$

This equation expresses that the optimal action-value should equal the immediate reward plus the discounted value of the best possible next action. This is the condition we would like our learning algorithm to approximate.

However, we do not know the true optimal action-values $q^*$. So we introduce a parameterised function $Q(s, a)$ to estimate them. Suppose we have observed $N$ transitions starting from $(s, a)$:

$$
(s, a, r^{(i)}, s'^{(i)}), \quad i = 1, \dots, N
$$

We define a squared loss function to encourage $Q(s,a)$ to match the Bellman optimality target:

$$
\mathcal{L}(s,a) = \frac{1}{2N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma \max_{a'} Q(s'^{(i)}, a') \right] \right)^2
$$

We do not index $a'$ because the $\max$ is computed over all possible actions in $s'^{(i)}$, not over the action that was actually taken. In Q-learning, we use the best available estimate, regardless of behaviour.

We now take the partial derivative of the loss with respect to $Q(s,a)$:

$$
\frac{\partial \mathcal{L}(s,a)}{\partial Q(s,a)} = \frac{1}{N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma \max_{a'} Q(s'^{(i)}, a') \right] \right)
$$

In writing this, we assume that the target values $\left[ r^{(i)} + \gamma \max_{a'} Q(s'^{(i)}, a') \right]$ are known and fixed. That is, we take the gradient only with respect to $Q(s,a)$.

To update $Q(s, a)$, we apply:

$$
\Delta Q(s, a) = -\eta \frac{\partial \mathcal{L}(s,a)}{\partial Q(s,a)}
$$

Substituting:

$$
\Delta Q(s, a) = -\eta \frac{1}{N} \sum_{i=1}^{N} \left( Q(s, a) - \left[ r^{(i)} + \gamma \max_{a'} Q(s'^{(i)}, a') \right] \right)
$$

In an online setting (i.e., $N = 1$), we suppress the sum and define the instantaneous update for a single transition $(s, a, r, s')$:

$$
\Delta Q(s, a) = -\eta \left( Q(s, a) - \left[ r + \gamma \max_{a'} Q(s', a') \right] \right)
$$

which simplifies to the Q-learning update rule:

$$
\Delta Q(s, a) = \eta \left( r + \gamma \max_{a'} Q(s', a') - Q(s, a) \right)
$$

</details>

## Differences between SARSA and Q-Learning

SARSA, as an on-policy reinforcement learning algorithm, directly learns the value of the policy being executed, including the exploration steps. It updates its Q-values based on the actual actions taken by the agent, according to the equation:
$$Q(s, a) \leftarrow Q(s, a) + \eta [r + \gamma Q(s', a') - Q(s, a)],$$
where $\eta$ is the learning rate, $r$ is the reward, $\gamma$ is the discount factor, $s$ is the current state, $a$ is the current action, $s'$ is the new state, and $a'$ is the new action according to the policy. This approach aligns SARSA's learning process closely with the agent's current strategy, leading to a more conservative policy development, especially in environments where certain actions might entail significant penalties.

Contrastingly, Q-Learning operates under an off-policy regime, aiming to learn the value function of the optimal policy independently of the agent's actions. It optimistically assumes the selection of the best possible future action, maximizing potential rewards:
$$Q(s, a) \leftarrow Q(s, a) + \eta [r + \gamma \max_{a'} Q(s', a') - Q(s, a)].$$
This difference in approach positions Q-Learning as a method capable of adopting a more aggressive exploration strategy, pursuing optimal future rewards without the direct influence of the agent's exploration strategy or the associated risks.

Theoretically, both SARSA and Q-Learning can converge to the optimal policy, especially as the exploration rate ($\epsilon$) decays appropriately over time, ensuring sufficient exploration and exploitation balance. However, practical challenges in reinforcement learning environments, such as the complexity of the state space, the variability of environmental dynamics, and the adequacy of the exploration decay rate, may hinder this convergence. Theoretical convergence assumes ideal conditions that often differ from real-world scenarios, where factors such as exploration strategy, reward distribution, and action penalties can significantly impact the learning process and the efficacy of the convergence to an optimal policy.


# Exercise

Assume the agent follows a greedy policy. How do SARSA and Q-learning differ in terms of action selection during learning, and how does this affect their Q-value updates?

<details>
<summary>Show Solution</summary>

Assume the same initial random Q-values, denoted as $Q_1$.

**SARSA:**  
In SARSA, the next action $a'$ is selected based on the current Q-values ($Q_1$) **before** the update.
- Action selection for the next state $s'$: $a' = \arg\max_{b} Q_1(s', b)$  
- Q-value update for the current state $s$ and action $a$:  
  $$
  Q_2(s, a) = Q_1(s, a) + \eta [r + \gamma Q_1(s', a') - Q_1(s, a)]
  $$

**Q-learning:**  
In Q-learning, the update uses the greedy value of the next state based on $Q_1$, without needing to select an actual action. Since the next action selection happens after the update (e.g., for executing the next step), it will be based on $Q_2$.
- Q-value update for the current state $s$ and action $a$:  
  $$
  Q_2(s, a) = Q_1(s, a) + \eta [r + \gamma \max_{b} Q_1(s', b) - Q_1(s, a)]
  $$
- If an action $a'$ is selected afterwards, it would be based on the updated values: $a' = \arg\max_{b} Q_2(s', b)$

**Key Difference:**  
SARSA updates its Q-values based on the action selected under the current policy **before** the update, so $a'$ is determined by $Q_1$.  
Q-learning does not select $a'$ as part of the update; instead, it uses the maximum over $Q_1(s', b)$ directly. If the agent follows a greedy policy and selects $a'$ afterward, it will do so using $Q_2$, creating a temporal distinction between the two methods.

</details>

## Need for Statistics

TD algorithms, including SARSA and Q-learning, are **stochastic**. Their performance depends on initialisation, the chosen policy, and the probabilistic nature of environment transitions and rewards.

We define an **episode** as the period during which we place the agent in the environment until it completes the task. A single episode contains many **steps**—individual state–action–reward transitions—and is typically not sufficient for the agent to learn the task. Therefore, the agent is usually exposed to many consecutive episodes. Across these episodes, the information stored in the Q-values is retained.

The number of episodes and steps required depends on the difficulty of the task and is usually chosen by the modeller. This full training process is termed a **run**.

To obtain reliable learning curves, it is necessary to average performance over multiple runs (i.e. several independent sets of episodes), each starting with different random initialisations. This averaging is again a modelling choice.

It's also crucial to consider the **variance** in outcomes across different runs. Due to the stochastic nature of both the algorithms and the environments they interact with, significant variability in learning outcomes can occur. Statistical analysis, such as computing confidence intervals or measuring variance, provides deeper insight into the **reliability and robustness** of the learned policy.

# Exercise

An agent operates in a 12 x 4 environment. Its starting position is located at coordinates $(3, 0)$, aiming to reach the goal at the opposite end at $(3, 11)$. This scenario introduces a "cliff" spanning from $(3, 1)$ to $(3, 10)$. Any step into this zone results in the agent being sent back to the start, receiving a large negative reward of $-500$. Reaching the target provides the agent with reward $+10$. The environment supports four actions: up, down, left, and right, represented numerically as $0, 1, 2,$ and $3$, respectively. The `step` function dictates the transition dynamics, including the penalties for falling off the cliff and the conditions for staying within the grid boundaries.

This scenario is inspired by an example in the book by Sutton and Barto on reinforcement learning. The `train` function encapsulates the learning process, employing an `epsilon_greedy_policy` to balance exploration and exploitation. The choice between SARSA and Q-Learning is determined by the `method` parameter.

Your task is to implement the missing parts in the code provided below and observe how they differ in addressing this problem. Discuss why this is the case.

**Tasks:**
1. Complete the `epsilon_greedy_policy` function to choose actions based on the current state and Q-values.
2. Complete the sections marked `TODO` for calculating the target for both SARSA and Q-Learning algorithms.
3. Update the Q-values based on the calculated targets.
4. Run the training process for both SARSA and Q-Learning methods.
5. Plot the average rewards per episode to compare the performance of the two methods.
6. What are the observed differences in how SARSA and Q-Learning handle the "cliff" environment? Consider why one method might perform better or differently than the other in this specific scenario.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class CliffWalkingEnv:
    def __init__(self):
        self.width = 12
        self.height = 4
        self.start = (3, 0)
        self.goal = (3, 11)
        self.cliff = [(3, i) for i in range(1, 11)]

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, state, action):
        moves = {0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)}  # Up, Down, Left, Right
        next_state = (state[0] + moves[action][0], state[1] + moves[action][1])
        if next_state in self.cliff:
            return self.start, -500, True
        if next_state == self.goal:
            return next_state, 10, True
        if 0 <= next_state[0] < self.height and 0 <= next_state[1] < self.width:
            return next_state, -1, False
        return state, -1, False

def epsilon_greedy_policy(Q, state, epsilon=0.1):
    if np.random.rand() < epsilon:
        return 0 #TO DO: replace with implementing the uniform random exploratory action (0,1,2,3)
    else:
        max_value = np.max(Q[state])
        best_actions = np.flatnonzero(Q[state] == max_value)
        return np.random.choice(best_actions)

def train(env, episodes, eta, gamma, epsilon, method):
    Q = np.zeros((env.height, env.width, 4))
    episode_rewards = []

    for _ in range(episodes):
        total_reward = 0
        state = env.reset()
        action = epsilon_greedy_policy(Q, state, epsilon)
        done = False

        while not done:
            next_state, reward, done = env.step(state, action)
            if method == 'sarsa':
                if not done:
                    next_action = epsilon_greedy_policy(Q, next_state, epsilon)
                    target = Q[next_state][next_action]
                    Q[state][action] += eta * (reward + gamma * target - Q[state][action])
                    state = next_state
                    action = next_action
                 else:
                    Q[state][action] += eta * (reward - Q[state][action])
            elif method == 'q_learning':
                if not done:
                    #TO DO: Implement Q-Learning
                    target = None #Replace None with the Q-value of the next state and the best action
                    #and complete the rest of the algorithm
                else:
                     Q[state][action] += 0 #Replace 0 with the correct update for the terminal state
            total_reward += reward
        episode_rewards.append(total_reward)

    return episode_rewards

env = CliffWalkingEnv()
episodes = 1000
eta = 0.1
gamma = 0.999
epsilon = 0.1

#TO DO: Uncomment the following lines to train the agent after implementing the algorithms
#sarsa_rewards = train(env, episodes, eta, gamma, epsilon, 'sarsa')
#q_learning_rewards = train(env, episodes, eta, gamma, epsilon, 'q_learning')

# Plotting
#TO DO: Plot the average reward as average of ten episodes to reduce noise
# Use np.mean with axis=1 after reshaping the rewards array into rows of 10 episodes each,
# then plot the average reward per group using plt.plot
# plt.xlabel('Episodes (grouped in tens)')
# plt.ylabel('Average Reward')
# plt.legend()
# plt.title('SARSA vs Q-Learning')
# plt.show()

<details>
<summary>Show Solution</summary>

```python
import numpy as np
import matplotlib.pyplot as plt

class CliffWalkingEnv:
    def __init__(self):
        self.width = 12
        self.height = 4
        self.start = (3, 0)
        self.goal = (3, 11)
        self.cliff = [(3, i) for i in range(1, 11)]

    def reset(self):
        self.state = self.start
        return self.state

    def step(self, state, action):
        moves = {0: (-1, 0), 1: (1, 0), 2: (0, -1), 3: (0, 1)}  # Up, Down, Left, Right
        next_state = (state[0] + moves[action][0], state[1] + moves[action][1])
        if next_state in self.cliff:
            return self.start, -500, True
        if next_state == self.goal:
            return next_state, 10, True
        if 0 <= next_state[0] < self.height and 0 <= next_state[1] < self.width:
            return next_state, -1, False
        return state, -1, False

def epsilon_greedy_policy(Q, state, epsilon=0.1):
    if np.random.rand() < epsilon:
        return np.random.randint(4)
    else:
        max_value = np.max(Q[state])
        best_actions = np.flatnonzero(Q[state] == max_value)
        return np.random.choice(best_actions)

def train(env, episodes, eta, gamma, epsilon, method):
    Q = np.zeros((env.height, env.width, 4))
    episode_rewards = []

    for _ in range(episodes):
        total_reward = 0
        state = env.reset()
        action = epsilon_greedy_policy(Q, state, epsilon)
        done = False

        while not done:
            next_state, reward, done = env.step(state, action)
            if method == 'sarsa':
                if not done:
                    next_action = epsilon_greedy_policy(Q, next_state, epsilon)
                    target = Q[next_state][next_action]
                    Q[state][action] += eta * (reward + gamma * target - Q[state][action])
                    state = next_state
                    action = next_action
                else:  
                    Q[state][action] += eta * (reward - Q[state][action])
            elif method == 'q_learning':
                if not done:
                    target = np.max(Q[next_state])
                    Q[state][action] += eta * (reward + gamma * target - Q[state][action])
                    state = next_state
                    action = epsilon_greedy_policy(Q, state, epsilon)
                else:
                     Q[state][action] += eta * (reward- Q[state][action])
            total_reward += reward
        episode_rewards.append(total_reward)

    return episode_rewards

env = CliffWalkingEnv()
episodes = 1000
eta = 0.1
gamma = 0.999
epsilon = 0.1

sarsa_rewards = train(env, episodes, eta, gamma, epsilon, 'sarsa')
q_learning_rewards = train(env, episodes, eta, gamma, epsilon, 'q_learning')

# Plotting
plt.plot(np.mean(np.array(sarsa_rewards).reshape(-1, 10), axis=1), label='SARSA')
plt.plot(np.mean(np.array(q_learning_rewards).reshape(-1, 10), axis=1), label='Q-Learning')
plt.xlabel('Episodes (grouped in tens)')
plt.ylabel('Average Reward')
plt.legend()
plt.title('SARSA vs Q-Learning')
plt.show()
```
</details>